In [28]:
# infile = 'dataset\description_0.docx'
infile = '..\\dataset\\requirements_0_reduced_6.docx'

In [53]:
from attr import dataclass
import docx

document = docx.Document(infile)

@dataclass
class HeadingInfo:
    text: str
    inheritance_number: int

class HeadingsTracker:
    def __init__(self):
        self.headings_stack = []
        pass

    def next_elem(self, elem) -> bool:
        HEADING_STYLE_NAME = "Heading"
        if isinstance(elem, docx.text.paragraph.Paragraph):
            style_name = elem.style.name
            if style_name.startswith(HEADING_STYLE_NAME):
                inheritance_number = int(style_name[len(HEADING_STYLE_NAME):])

                # remove lover headings from stack
                while(len(self.headings_stack) > 0 and self.headings_stack[-1].inheritance_number >= inheritance_number):
                    del self.headings_stack[-1]

                if len(elem.text) > 0: 
                    self.headings_stack.append(HeadingInfo(elem.text, inheritance_number))
                return True
        return False

    def get_text(self):
        s = ""
        for info in self.headings_stack:
            s += '#'*info.inheritance_number + ' ' + info.text + '\n'
        return s

htracker = HeadingsTracker()

# for paragraph in document.paragraphs:
text_parts = []
cur_heading = ''
cur_body = ''
prev_is_normal = False
for paragraph in document.iter_inner_content():
    cur_is_normal = False
    if isinstance(paragraph, docx.text.paragraph.Paragraph):
        if not htracker.next_elem(paragraph):
            if not prev_is_normal or len(paragraph.text) > 200:
                if len(cur_body) > 0:
                    text_parts.append(cur_heading + cur_body)
                cur_heading = htracker.get_text()
                cur_body = ''
            if len(paragraph.text) > 0 and not paragraph.text.isspace():
                cur_body += paragraph.text + '\n'
                cur_is_normal = True
    elif isinstance(paragraph, docx.table.Table):
        for row in paragraph.rows:
            for cell in row.cells:
                cur_body += f"| {cell.text} "
            cur_body += "|\n"

    prev_is_normal = cur_is_normal


In [54]:
text_parts_clear = []
for part in text_parts:
    if len(part) > 0:
        text_parts_clear.append(part)
print("\n".join(text_parts_clear))

Общие технические требования

Содержание

# Введение
Использование настоящего стандарта в конкретных условиях может потребовать учета дополнительных или специфических требований.

# Введение
Настоящий стандарт является модифицированным по отношению к международному стандарту [1] (Насосы центробежные герметичные для технологического обслуживания в нефтяной, нефтехимической и газовой промышленности).

# Введение
Дополнительные положения и требования, а также сноски, включенные в текст настоящего стандарта для учета потребностей национальной экономики и особенностей российской национальной стандартизации, выделены курсивом.

# Введение
Знак (●) в начале параграфа или его раздела указывает на то, что здесь требуется принятие решения или представление заказчиком дополнительной информации. Такую информацию необходимо привести в опросных листах или указать в запросе, или в заказе на поставку.
Из соображений удобства и в информационных целях в настоящем стандарте в скобках после величин в сист

In [57]:
# measure max length
max_len = 0
max_text = ''
for part in text_parts_clear:
    if (len(part) > max_len):
        max_len = max(max_len, len(part))
        max_text = part
print(max_len)
print(max_text)

5791
# Требования по электробезопасности
Продолжение таблицы 2
| Организация стандартизации | Стандарты | Применение | Обозначение
классификации опасной электрической зоны |
| NEC®d | NFPA 70

Разделы 500-502, 504 | Соединенные	Штаты Америки | Класс,	подразделение, группа, температурный
класс |
| NEC®e | NFPA 70

Раздел 505 | Соединенные	Штаты Америки | Класс I, зона, группа газов, температурный
класс |
| CEC®f | CSA C22.1-06, Раздел 18 | Канада | Основная: Принятие МЭК - зона, группа газов, температурный класс Вспомогательная: класс, подразделение, группа, температурный
класс |
| a - МЭК: Международная электротехническая комиссия

b - CENELEC: Европейский комитет по электротехнической стандартизации

c - ATEX: “Взрывоопасные среды”, “Оборудование, предназначенное для использования во взрывоопасных средах”
d-	NEC®: Национальный электротехнический кодекс®, опубликованный Национальной ассоциацией противопожарной защиты (NFPA)
e - CEC®: Канадский электрический кодекс®, опубликованный Кана